## Import Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 35.1 MB/s 
     |████████████████████████████████| 7.6 MB 51.2 MB/s 
     |████████████████████████████████| 182 kB 76.7 MB/s 


In [ ]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 240 kB 32.6 MB/s 
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=83c675bd3c95c68ef902e0e3715c962cf5c4ada4fc0b6b52cdbeaea7fbe52653
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import transformers
import emoji as emoji
import re
from torch.utils.data import TensorDataset
from urllib.parse import urlparse
import string
import torch
from nltk.corpus import stopwords
import gc
import torch.nn as nn
from transformers import AutoModel
from transformers import BertModel, BertTokenizer
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification
from imblearn.over_sampling import SMOTE

In [ ]:
# specify GPU
device = torch.device("cuda")

Load & Visualize the Strongly Classified Tweets Dataset

In [ ]:
df= pd.read_csv('/content/drive/MyDrive/Cleaned_MBTI.csv',index_col=[0]) 
df 

,type,posts,E-I,N-S,F-T,J-P,Cleaned_posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,I,N,F,J,and moments sportscenter not top ten plays pra...
1,ENTP,'I'm finding the lack of me in these posts ver...,E,N,T,P,im finding the lack of me in these posts very ...
2,INTP,'Good one _____ https://www.youtube.com/wat...,I,N,T,P,good one of course to which i say i know thats...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",I,N,T,J,dear i enjoyed our conversation the other day ...
4,ENTJ,'You're fired.|||That's another silly misconce...,E,N,T,J,youre fired thats another silly misconception ...
...,...,...,...,...,...,...,...
8670,ISFP,'https://www.youtube.com/watch?v=t8edHB_h908||...,I,S,F,P,ixfp just because i always think of cats as fi...
8671,ENFP,'So...if this thread already exists someplace ...,E,N,F,P,soif this thread already exists someplace else...
8672,INTP,'So many questions when i do these things. I ...,I,N,T,P,so many questions when i do these things i wou...
8673,INFP,'I am very conflicted right now when it comes ...,I,N,F,P,i am very conflicted right now when it comes t...


In [ ]:
df=df[['Cleaned_posts','type','E-I','N-S',	'F-T',	'J-P'	]]

In [ ]:
count_df=df.groupby('type')['Cleaned_posts'].count().reset_index()
count_df

,type,Cleaned_posts
0,ENFJ,190
1,ENFP,673
2,ENTJ,229
3,ENTP,683
4,ESFJ,42
5,ESFP,48
6,ESTJ,39
7,ESTP,89
8,INFJ,1470
9,INFP,1827


In [ ]:
fig = px.bar(count_df, x="type", y='Cleaned_posts',title="Class Count")
fig.update_layout(width=500, height=400)
fig.show()

In [ ]:
fig_pie = px.pie(count_df, values='Cleaned_posts', names='type',width=500, height=400)
fig_pie.show()

In [ ]:
#df= df.reindex(columns=['Cleaned_posts','type'])
df= df.rename(columns={'type': 'label'})
df

,Cleaned_posts,label,E-I,N-S,F-T,J-P
0,and moments sportscenter not top ten plays pra...,INFJ,I,N,F,J
1,im finding the lack of me in these posts very ...,ENTP,E,N,T,P
2,good one of course to which i say i know thats...,INTP,I,N,T,P
3,dear i enjoyed our conversation the other day ...,INTJ,I,N,T,J
4,youre fired thats another silly misconception ...,ENTJ,E,N,T,J
...,...,...,...,...,...,...
8670,ixfp just because i always think of cats as fi...,ISFP,I,S,F,P
8671,soif this thread already exists someplace else...,ENFP,E,N,F,P
8672,so many questions when i do these things i wou...,INTP,I,N,T,P
8673,i am very conflicted right now when it comes t...,INFP,I,N,F,P


In [ ]:
df= df.rename(columns={'Cleaned_posts': 'posts'})
df.head(5)

,posts,label,E-I,N-S,F-T,J-P
0,and moments sportscenter not top ten plays pra...,INFJ,I,N,F,J
1,im finding the lack of me in these posts very ...,ENTP,E,N,T,P
2,good one of course to which i say i know thats...,INTP,I,N,T,P
3,dear i enjoyed our conversation the other day ...,INTJ,I,N,T,J
4,youre fired thats another silly misconception ...,ENTJ,E,N,T,J


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['label'] = le.fit_transform(df.label.values)

In [ ]:
df['E-I'] = le.fit_transform(df['E-I'].values)
df['N-S'] = le.fit_transform(df['N-S'].values)
df['F-T'] = le.fit_transform(df['F-T'].values)
df['J-P'] = le.fit_transform(df['J-P'].values)

In [ ]:
df = df.sample(frac=1)
df.head()

,posts,label,E-I,N-S,F-T,J-P
3262,it sounds like a very stressful situation i wo...,15,1,1,1,1
6312,or when someone bails last minute or even wors...,14,1,1,1,0
7925,it is a myth that fptypes are incessantly bad ...,2,0,0,1,0
6862,okay i edited my post to type the collage than...,0,0,0,0,0
2402,flare hi everybody i dont know what to say but...,9,1,0,0,1


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
df['posts'] = df['posts'].astype('U') # which will by default set the length to the max len it encounters
#df['Cleaned_posts']= df['Cleaned_posts'].astype('str') 
#df['Cleaned_posts'].dtype

In [47]:
List_code=['E-I','N-S',	'F-T','J-P']

In [66]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 16

In [60]:
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score,classification_report

In [61]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [62]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    total_correct = 0
    total = 0
    for label in np.unique(labels_flat):
      y_preds = preds_flat[labels_flat==label]
      y_true = labels_flat[labels_flat==label]
      a = len(y_preds[y_preds==label])
      b = len(y_true)
      total_correct += a
      total += b
      #print(f'Class: {label_dict_inverse[label]}')
      print(f'Accuracy:{a}/{b}\n')
      print(f'Overall accuracy: {total_correct/total:.2f}\n')

In [63]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [67]:
for x in List_code:
  print('---------------',x,'---------------',)
  train_text, temp_text, train_labels, temp_labels = train_test_split(df['posts'], df[x], 
                                                                    random_state=42, 
                                                                    test_size=0.3, 
                                                                    stratify=df[x])
  # will use temp_text and temp_labels to create validation and test set
  val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=42, 
                                                                test_size=0.7, 
                                                                stratify=temp_labels)
  
  #----------
  train_text= train_text.astype('string')
  val_text= val_text.astype('string')
  test_text= test_text.astype('string')

  #----------

  # tokenize and encode sequences in the training set
  
  tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = 280,
    padding=True,
    truncation=True,
    return_token_type_ids=False)
  
  # tokenize and encode sequences in the validation set
  tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = 280,
    padding=True,
    truncation=True,
    return_token_type_ids=False)
  
  tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = 280,
    padding=True,
    truncation=True,
    return_token_type_ids=False)
  
  #----------


  # for train set
  train_input_ids = torch.tensor(tokens_train['input_ids'])
  train_mask = torch.tensor(tokens_train['attention_mask'])
  train_labels = torch.tensor(train_labels.tolist())
  
  # for validation set
  val_input_ids  = torch.tensor(tokens_val['input_ids'])
  val_mask = torch.tensor(tokens_val['attention_mask'])
  val_labels = torch.tensor(val_labels.tolist())
  
  # for test set
  test_input_ids  = torch.tensor(tokens_test['input_ids'])
  test_mask = torch.tensor(tokens_test['attention_mask'])
  test_labels = torch.tensor(test_labels.tolist())


  #-------

  dataset_train = TensorDataset(train_input_ids , train_mask, train_labels)
  dataset_val = TensorDataset(val_input_ids, val_mask, val_labels)
  dataset_test = TensorDataset(test_input_ids, test_mask,test_labels)
  len(dataset_train), len(dataset_val),len(dataset_test)

  #-------

  model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels= df[x].nunique(),
                                                      output_attentions=False,
                                                      output_hidden_states=False)
  
  #-------
  dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)
  
  dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)
  
  dataloader_test = DataLoader(dataset_test,
                                    sampler=SequentialSampler(dataset_test),                
                                    batch_size=batch_size)

  #-------

  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model.to(device)

  epochs = 3

  optimizer = AdamW(model.parameters(),
                  lr=5e-5, 
                  eps=1e-8)
  #recommanded epsilon : 1e-8
  
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0,num_training_steps=len(dataloader_train)*epochs)

  

  for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    #torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')
    print("Accuracy of BERT is " + str(accuracy_score(list(true_vals), preds)))
    #accuracy_per_class(predictions, true_vals)

--------------- E-I ---------------


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/380 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.538354998787767
Validation loss: 0.562277939854836
F1 Score (Weighted): 0.6688963210702341
Accuracy of BERT is 0.7692307692307693


...


Epoch 1
Training loss: 0.673831615008806
Validation loss: 0.6418054870196751
F1 Score (Weighted): 0.6251130754947549
Accuracy of BERT is 0.6038461538461538


Epoch 2:   0%|          | 0/380 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.615575132401366
Validation loss: 0.6413461456493456
F1 Score (Weighted): 0.6194688952910944
Accuracy of BERT is 0.6038461538461538


Epoch 3:   0%|          | 0/380 [00:00<?, ?it/s]


Epoch 3
Training loss: 0.4953241257683227
Validation loss: 0.7255839358787147
F1 Score (Weighted): 0.6092249411895183
Accuracy of BERT is 0.6038461538461538


In [ ]:
'''
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    #torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')
'''

"\nfor epoch in tqdm(range(1, epochs+1)):\n    \n    model.train()\n    \n    loss_train_total = 0\n\n    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)\n    for batch in progress_bar:\n\n        model.zero_grad()\n        \n        batch = tuple(b.to(device) for b in batch)\n        \n        inputs = {'input_ids':      batch[0],\n                  'attention_mask': batch[1],\n                  'labels':         batch[2],\n                 }       \n\n        outputs = model(**inputs)\n        \n        loss = outputs[0]\n        loss_train_total += loss.item()\n        loss.backward()\n\n        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)\n\n        optimizer.step()\n        scheduler.step()\n        \n        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})\n         \n        \n    #torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')\n        \n    tqdm

In [ ]:
'''
_, predictions, true_vals = evaluate(dataloader_validation)
print('predictions:', predictions)
print('true_vals:', true_vals)
'''

In [ ]:
'''
from sklearn.metrics import accuracy_score,classification_report
print("Performance:")
# model's performance
print('Classification Report')
preds = np.argmax(predictions, axis=1)
print(classification_report(true_vals, preds))
print("Accuracy of BERT is " + str(accuracy_score(true_vals, preds)))
'''

In [ ]:
#accuracy_per_class(predictions, true_vals)